In [2]:
##IMPORTING REQUIRED LIBRARIES

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime

In [3]:
## FUCTION THAT SCRAPES REQUIRED DATA FROM THE WEBSITE USING BeautifulSoup AND requests LIB
def scrape_data(link):
    
    html = requests.get(link).text
    scrape = BeautifulSoup(html,'lxml')
    
    ## LISTS THAT STORES THE SCRAPPED DATA
    
    DATA_array = []
    temp_lis = []
    rows = []
    
    ## NAMING OF THE DATA ATTRIBUTES (COLUMMBS)
    
    columbs = ['high','low','last_price','prev_close','change','gain','volume','v1','v2','v3','30DMA','50DMA',
    '150DMA','200DMA','P/E','P/B','5_DAY_PERFO','DELIVERY_AVG','DEL','DEL','DEL','UP_CIR',
    'LO_CIR','VWAp']
    
    ## SCRAPPING USING BeautifulSoup AND STORING IT IN THE LISTS
    
    table = scrape.find_all('div', class_ = 'bsr_table hist_tbl_hm')
    for t in table:
     bodyy = t.find_all('tbody')
     for tts in bodyy:
      stock_row = tts.find_all('tr')
      for st in stock_row:
        bt = st.find_all('td', {'align': 'right'})
        title = st.find('a')
        if title == None:
          rows.append("empty")
        else:
          rows.append(title.text)
        for atr in bt:
          value = atr.text
          value = value.replace('-','0')
          temp_lis.append(value)
        DATA_array.append(temp_lis)
        temp_lis = []
        
    ## CREATING AN PANDAS DATA FRAME FROM THE SCRAPPED STOCK DATA
    
    pd.set_option("display.max_columns", None)
    df = pd.DataFrame(DATA_array,rows,columbs)
    return df

In [4]:
# SCRAPPING THE ENTIRE LIVE NIFTY 500 RAW DATA FROM MONEYCONTROL WEBSITES

# NIFTY 500 STOCKS IN UPTREND (FOR THE DAY)
gain = scrape_data('https://www.moneycontrol.com/stocks/marketstats/nse-gainer/nifty-500_7/')

# NIFTY 500 STOCKS IN DOWNTREND (FOR THE DAY)
lose = scrape_data('https://www.moneycontrol.com/stocks/marketstats/nse-loser/nifty-500_7/')

# MERGING BOTH THE DATAFRAMES TOGETHER
df = pd.concat([gain,lose])

In [5]:
## CLEANING AND FORMATTING THE RAW DATA OBTAINED TO MAKE IT READABLE

## DROPING DATA ATTRIBUTES IRRELEVANT FOR METHOD USED
df.drop(['empty'], axis = 0, inplace = True)
df.drop(['DEL','DELIVERY_AVG','volume','v1','v2','v3','5_DAY_PERFO','change','30DMA','50DMA','150DMA','200DMA','P/E','P/B','VWAp'], axis = 1, inplace = True)
    
    ## CHANGING THE DATA TYPES OF ATTRIBUTES FROM STRING TO FLOAT TO MAKE IT SUITABE FOR CALCULATIONS 
for dat in df.columns:
    df[dat] = df[dat].str.replace(',', '').replace('-', '0').astype(float)
    

In [6]:
##CALCULATION PIVOT POINTS TO HELP IN ANALYSIS AND STUDY THE MOVEMENT OF STOCK PRICES
    
df['PIVOT'] = (df['high']+df['low']+df['last_price'])/3   # (High + Low + Close) / 3
df['R1'] = (2*(df['PIVOT']))-df['low']                    # 2 * PP - Low
df['R2'] = df['PIVOT']+(df['high']-df['low'])             # PP + (High - Low)
df['R3'] = df['high']+(2*(df['PIVOT']-df['low']))         # high + 2 * (pivot - Low)
df['R4'] = df['high'] + (3 * (df['PIVOT']-df['low']))     # H + 3 x (PP - L)
df['S1'] = (2*df['PIVOT']) - df['high']                   # 2 * PP - High
df = df.round(decimals=2)

df

,high,low,last_price,prev_close,gain,UP_CIR,LO_CIR,PIVOT,R1,R2,R3,R4,S1
Hatsun Agro,1318.85,1203.30,1291.95,1228.75,5.14,1474.50,983.00,1271.37,1339.43,1386.92,1454.98,1523.05,1223.88
Bajaj Electric,1333.80,1219.45,1312.05,1259.45,4.18,1511.30,1007.60,1288.43,1357.42,1402.78,1471.77,1540.75,1243.07
Wipro,676.80,650.10,670.80,644.30,4.11,708.70,579.90,665.90,681.70,692.60,708.40,724.20,655.00
Blue Star,1020.00,983.00,1015.25,985.30,3.04,1182.35,788.25,1006.08,1029.17,1043.08,1066.17,1089.25,992.17
Persistent,4752.10,4551.05,4640.15,4503.70,3.03,4954.05,4053.35,4647.77,4744.48,4848.82,4945.53,5042.25,4543.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hindalco,457.00,444.75,450.15,450.50,0.08,495.55,405.45,450.63,456.52,462.88,468.77,474.65,444.27
Mindtree,4700.00,4570.55,4587.70,4591.05,0.07,5050.15,4131.95,4619.42,4668.28,4748.87,4797.73,4846.60,4538.83
GAIL,130.20,127.65,129.50,129.55,0.04,142.50,116.60,129.12,130.58,131.67,133.13,134.60,128.03
DCM Shriram,1014.50,982.00,994.10,994.40,0.03,1193.25,795.55,996.87,1011.73,1029.37,1044.23,1059.10,979.23


In [7]:
##SAVING IT INTO AN EXCEL
df.to_csv('11-B.csv')